In [14]:
import numpy as np
def get_matrix_dim_from_n_lower_tri(n_lower_tri: int)-> int:
    n = (np.sqrt(8 * n_lower_tri + 1) - 1) / 2
    if n.is_integer():
        return int(n)
    else:
        raise ValueError(f"The input {n_lower_tri} does not correspond to a valid square matrix dimension.")

# print(get_matrix_dim_from_n_lower_tri(6))
def create_symmetric_matrix_from_array(a):
    n = get_matrix_dim_from_n_lower_tri(len(a))
    A = np.zeros((n,n))
    A[np.tril_indices(n)] = a
    A = A + A.T - np.diag(A.diagonal())
    return A

def get_n_lower_tri_from_matrix_dim(n: int)-> int:
    return int(n*(n+1)/2)

print(get_n_lower_tri_from_matrix_dim(6))
a = np.arange(0,15)
print(a)
tril = np.tril(a)
print(create_symmetric_matrix_from_array(a))

21
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[[ 0.  1.  3.  6. 10.]
 [ 1.  2.  4.  7. 11.]
 [ 3.  4.  5.  8. 12.]
 [ 6.  7.  8.  9. 13.]
 [10. 11. 12. 13. 14.]]


In [20]:
from pydrake.all import (DecomposeAffineExpressions, Variable)

def matrix_inverse_pydrake(matrix, vars):
    n = matrix.shape[0]
    assert matrix.shape == (n, n), "Input matrix must be square"
    
    # Convert input matrix to pydrake symbolic Expression
    matrix = DecomposeAffineExpressions(matrix, vars)
    
    # Compute inverse using pydrake's .inverse() method
    inverse_matrix = matrix.inverse()
    
    return inverse_matrix

x = Variable("x")
y = Variable("y")

matrix = np.array([[x+0, y+0], [1, x+y]])
matrix = DecomposeAffineExpressions(matrix, [x,y])
inverse_matrix = matrix.inverse

print(inverse_matrix)

TypeError: DecomposeAffineExpressions(): incompatible function arguments. The following argument types are supported:
    1. (expressions: numpy.ndarray[object[m, 1]], vars: numpy.ndarray[object[m, 1]]) -> Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]]]
    2. (v: numpy.ndarray[object[m, 1]]) -> Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[object[m, 1]]]

Invoked with: array([[<Expression "x">, <Expression "y">],
       [1, <Expression "(x + y)">]], dtype=object), [Variable('x', Continuous), Variable('y', Continuous)]

In [21]:
import numpy as np

def matrix_inverse(matrix):
    n = matrix.shape[0]
    assert matrix.shape == (n, n), "Input matrix must be square"
    
    augmented_matrix = np.hstack((matrix, np.eye(n)))
    for i in range(n):
        # Find pivot
        max_row = i + np.argmax(np.abs(augmented_matrix[i:, i]))
        # Swap rows
        augmented_matrix[[i, max_row]] = augmented_matrix[[max_row, i]]
        # Normalize pivot row
        augmented_matrix[i] /= augmented_matrix[i, i]
        # Eliminate other rows
        for j in range(n):
            if j != i:
                augmented_matrix[j] -= augmented_matrix[j, i] * augmented_matrix[i]
    return augmented_matrix[:, n:]

In [36]:
def matrix_inverse(A):
    n = len(A)
    A = A.tolist()
    I = np.identity(n).tolist()
    
    for fd in range(n):
        fdScaler = 1.0 / A[fd][fd]
        for j in range(n):
            A[fd][j] = A[fd][j] * fdScaler
            I[fd][j] = I[fd][j] * fdScaler
        for i in list(range(n))[0:fd] + list(range(n))[fd+1:]:
            crScaler = A[i][fd]
            for j in range(n):
                A[i][j] = A[i][j] - crScaler * A[fd][j]
                I[i][j] = I[i][j] - crScaler * I[fd][j]
    return np.array(I)

In [33]:
from pydrake.all import (MathematicalProgram, MonomialBasis)
prog = MathematicalProgram()
deg = 2
x = prog.NewContinuousVariables(2, "x")
y = prog.NewContinuousVariables(2, "y")
v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)]

In [34]:

import random
import numpy as np

def get_minor_n_cofactor_matrix(matrix):
    minor_matrix = np.zeros(matrix.shape)
    cofactor_matrix = np.zeros(matrix.shape)
    
    matrix_rows, matrix_cols = matrix.shape
    for row in range(matrix_rows):
        for col in range(matrix_cols):
            minor_sub_matrix = np.delete(matrix, row, axis=0)           # delete ith row
            minor_sub_matrix = np.delete(minor_sub_matrix, col, axis=1) # delete jth column
            minor = np.linalg.det(minor_sub_matrix)                     # determinant of submatrix
            minor_matrix[row, col] = minor
            cofactor_matrix[row, col] = (-1)**(row+col) * minor         # cofactor from minor
    return minor_matrix, cofactor_matrix

def get_determinant(matrix, cofactor_matrix):
    matrix_rows, matrix_cols = matrix.shape
    is_any_row = bool(random.getrandbits(1))        # to enter calculation by row or column randomnly
    rand_idx = random.randint(0, matrix_rows-1)     # to get any row or any col by random
    
    if is_any_row:
        determinant = sum(matrix[rand_idx, :] * cofactor_matrix[rand_idx, :]) # any row 
    else: 
        determinant = sum(matrix[:, rand_idx] * cofactor_matrix[:, rand_idx]) # any column 
    
    return determinant

def matrix_inverse(matrix):
    matrix_rows, matrix_cols = matrix.shape
    assert matrix_rows == matrix_cols, "Input matrix must be square"
    
    minor_matrix, cofactor_matrix = get_minor_n_cofactor_matrix(matrix)
    determinant = get_determinant(matrix, cofactor_matrix)
    
    inverse_matrix = (1/determinant) * cofactor_matrix.T
    
    return inverse_matrix

In [ ]:
def smaller_matrix(original_matrix,row, column):
    for ii in range(len(original_matrix)):
        new_matrix=np.delete(original_matrix,ii,0)
        new_matrix=np.delete(new_matrix,column,1)
        return new_matrix


def determinant(matrix):
    """Returns a determinant of a matrix by recursive method."""
    (r,c) = matrix.shape 
    if r != c:
        print("Error!Not a square matrix!")
        return None
    elif r==2:
        simple_determinant = matrix[0][0]*matrix[1][1]-matrix[0][1]*matrix[1][0]
        return simple_determinant
    else: 
        answer=0
        for j in range(r):
            cofactor = (-1)**(0+j) * matrix[0][j] * determinant(smaller_matrix(matrix, 0, j))
            answer+= cofactor
        return answer

In [10]:
def transposeMatrix(m):
    return list(map(list,zip(*m)))

def getMatrixMinor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def getMatrixDeternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*getMatrixDeternminant(getMatrixMinor(m,0,c))
    return determinant

def getMatrixInverse(m):
    if isinstance(m, np.ndarray):
        m = m.tolist()
    determinant = getMatrixDeternminant(m)
    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = getMatrixMinor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * getMatrixDeternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = transposeMatrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return np.array(cofactors)

In [11]:
import numpy as np
a = np.array([[1,2,-1],[2,1,2],[-1,2,1]])
print(a)
print(np.linalg.inv(a))
print(getMatrixInverse(a))
a = np.array([[5., 3., 1.], [3., 9., 4.], [1., 3., 5.]])
print(np.linalg.inv(a))
print(getMatrixInverse(a))

[[ 1  2 -1]
 [ 2  1  2]
 [-1  2  1]]
[[ 0.1875  0.25   -0.3125]
 [ 0.25    0.      0.25  ]
 [-0.3125  0.25    0.1875]]
[[ 0.1875  0.25   -0.3125]
 [ 0.25   -0.      0.25  ]
 [-0.3125  0.25    0.1875]]
[[ 2.50000000e-01 -9.09090909e-02  2.27272727e-02]
 [-8.33333333e-02  1.81818182e-01 -1.28787879e-01]
 [ 2.01858732e-18 -9.09090909e-02  2.72727273e-01]]
[[ 0.25       -0.09090909  0.02272727]
 [-0.08333333  0.18181818 -0.12878788]
 [ 0.         -0.09090909  0.27272727]]


In [38]:
a = np.array([[1,2,-1],[2,1,2],[-1,2,1]])
print(a)
print(np.linalg.inv(a))
print(matrix_inverse(a))
a = np.array([[5., 3., 1.], [3., 9., 4.], [1., 3., 5.]])
print(np.linalg.inv(a))
print(matrix_inverse(a))

[[ 1  2 -1]
 [ 2  1  2]
 [-1  2  1]]
[[ 0.1875  0.25   -0.3125]
 [ 0.25    0.      0.25  ]
 [-0.3125  0.25    0.1875]]
[[ 0.1875  0.25   -0.3125]
 [ 0.25    0.      0.25  ]
 [-0.3125  0.25    0.1875]]
[[ 2.50000000e-01 -9.09090909e-02  2.27272727e-02]
 [-8.33333333e-02  1.81818182e-01 -1.28787879e-01]
 [ 2.01858732e-18 -9.09090909e-02  2.72727273e-01]]
[[ 0.25       -0.09090909  0.02272727]
 [-0.08333333  0.18181818 -0.12878788]
 [ 0.         -0.09090909  0.27272727]]
